In [1]:
###IN PROGRESS

#Generative Adversarial Network on MNIST

In [2]:
#imports
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

In [3]:
#importing mnist dataset as a tf dataset
mnist = input_data.read_data_sets("../Data/MNIST/", one_hot=True)

Extracting ../Data/MNIST/train-images-idx3-ubyte.gz
Extracting ../Data/MNIST/train-labels-idx1-ubyte.gz
Extracting ../Data/MNIST/t10k-images-idx3-ubyte.gz
Extracting ../Data/MNIST/t10k-labels-idx1-ubyte.gz


In [4]:
#helper to sample from random uniform distribution
def sample_z(m, n):
    return np.random.uniform(-1.0, 1.0, size=[m,n])

In [5]:
#inputs and layers
x = tf.placeholder("float", shape=[None, 784]) #input to discriminator
z = tf.placeholder ("float", shape=[None, 100]) #input to generator

In [6]:
#####################
###   GENERATOR   ###
#####################

#100 -> 128 -> 784 (MNIST image)
g_w1 = tf.Variable(tf.random_normal([100, 128]))
g_b1 = tf.Variable(tf.zeros(shape=[128]))

g_w2 = tf.Variable(tf.random_normal([128, 784]))
g_b2 = tf.Variable(tf.zeros(shape=[784]))

#for backprop
theta_g = [g_w1, g_b1, g_w2, g_b2]

#construct generator
def generator(z):
    g_h1 = tf.nn.relu(tf.matmul(z, g_w1) + g_b1)
    g_logistic = tf.matmul(g_h1, g_w2) + g_b2
    g_prob = tf.nn.sigmoid(g_logistic)
    
    return g_prob

In [7]:
#####################
### DISCRIMINATOR ###
#####################

#784 -> 128 -> 1 
d_w1 = tf.Variable(tf.random_normal(shape=[784, 128]))
d_b1 = tf.Variable(tf.zeros(shape=128))

d_w2 = tf.Variable(tf.random_normal(shape=[128, 1]))
d_b2 = tf.Variable(tf.zeros(shape=1))

#for backprop
theta_d = [d_w1, d_w2, d_b1, d_b2]

#construct discriminator
def discriminator(x):
    '''feedforward discriminator'''
    d_h1 = tf.nn.relu(tf.matmul(x, d_w1) + d_b1)
    d_logistic = tf.matmul(d_h1, d_w2) + d_b2
    d_prob = tf.nn.sigmoid(d_logistic) #probability that input is real
    
    return d_prob, d_logistic

In [8]:
########################
### DC DISCRIMINATOR ###
########################

#784 -> conv -> conv -> 128 -> 1
dc_d_w1 = tf.Variable(tf.random_normal([5, 5, 1, 16]))
dc_d_b1 = tf.Variable(tf.zeros(shape=[16]))

dc_d_w2 = tf.Variable(tf.random_normal([3, 3, 16, 32]))
dc_d_b2 = tf.Variable(tf.zeros(shape=[32]))

dc_d_w3 = tf.Variable(tf.random_normal([7*7*32, 128]))
dc_d_b3 = tf.Variable(tf.zeros(shape=[128]))

dc_d_w4 = tf.Variable(tf.random_normal([128, 1]))
dc_d_b4 = tf.Variable(tf.zeros(shape=[1]))

#for backprop
theta_dc_c = [dc_d_w1, dc_d_b1, dc_d_w2, dc_d_b2, dc_d_w3, dc_d_b3, dc_d_w4, dc_d_b4]

#construct deep convolutional discriminator
def dc_discriminator(x):
    '''deep convolutional feedforward discriminator'''
    x = tf.reshape(x, shape = [-1, 28, 28, 1])
    conv1 = tf.nn.relu(tf.nn.conv2d(x, dc_d_w1, strides=[1, 2, 2, 1], padding="SAME") + dc_d_b1)
    conv2 = tf.nn.relu(tf.nn.conv2d(conv1, dc_d_w2, strides=[1, 2, 2, 1], padding='SAME') + dc_d_b2)
    conv2_out = tf.reshape(conv2, shape=[-1, 7*7*32])
    h3 = tf.nn.relu(tf.matmul(conv2_out, dc_d_w3) + dc_d_b3)
    dc_logistic = tf.matmul(h3, dc_d_w4) + dc_d_b4
    dc_prob = tf.nn.sigmoid(dc_logistic)
    
    return dc_prob, dc_logistic

In [15]:
#connection between discriminator and generator
g_sample = generator(z)
d_real, d_logit_real = dc_discriminator(x)
d_fake, d_logit_fake = dc_discriminator(g_sample)


#losses for D and G (d loss optimizes on true posities, false negatives)
    #cross entropy loss from logits (passing in data and labels all ones)
d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(d_logit_real, tf.ones_like(d_logit_real)))
    #cross entropy loss from logits (passing in data and labels all zeros)
d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(d_logit_fake, tf.zeros_like(d_logit_fake)))
    #account for both losses in total discriminator loss
d_loss = d_loss_real + d_loss_fake
    #generator is optimizing how many misclassifications is can cause
g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(d_logit_fake, tf.ones_like(d_logit_fake)))


#optimizers for D and G
d_solver = tf.train.AdamOptimizer().minimize(d_loss)#, var_list=theta_d)
g_solver = tf.train.AdamOptimizer().minimize(g_loss)#, var_list=theta_g)

In [16]:
batch_size = 128
z_dim = 100

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    i = 0
    for it in range(10000):
        #optimize G one batch (pass it noise)
        #optimize D one batch (pass it G output and real data)
    
        #grab batches; don't need labels!
        x_mb, _ = mnist.train.next_batch(batch_size)
        
        #train discriminator
        _, d_loss_cur = sess.run([d_solver, d_loss], feed_dict={x: x_mb, z: sample_z(batch_size, z_dim)})
        
        #train generator
        _, g_loss_cur = sess.run([g_solver, g_loss], feed_dict={z: sample_z(batch_size, z_dim)})
        
        
        if it % 100 == 0:
            print('Iteration: ', it)
            print('Generator loss: ', g_loss_cur)
            print('Discriminator loss: ', d_loss_cur)
            print()

Iteration:  0
Generator loss:  176.097
Discriminator loss:  4079.14

Iteration:  100
Generator loss:  259.463
Discriminator loss:  2077.9

Iteration:  200
Generator loss:  369.974
Discriminator loss:  2134.63

Iteration:  300
Generator loss:  265.548
Discriminator loss:  2217.59

Iteration:  400
Generator loss:  227.238
Discriminator loss:  2179.32

